In [1]:
!pip install image_classifiers
#!pip install keras==3.1.1 tensorflow==2.16.1
import numpy as np
import tensorflow as tf
from keras.applications import ResNet50V2
from keras.datasets import cifar100
from keras import Sequential, Input
from keras.layers import Dense, Dropout, RandomFlip, RandomTranslation, RandomRotation,RandomBrightness, RandomContrast, RandomZoom, GlobalAveragePooling2D
from keras.applications.resnet_v2 import preprocess_input
from keras.models import Model
from classification_models.keras import Classifiers
from keras.optimizers import SGD
from keras.activations import linear
from keras.utils import Progbar
from keras.backend import clear_session
from tensorflow.nn import softmax_cross_entropy_with_logits
import os

2024-04-25 14:56:09.166991: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-25 14:56:09.246076: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-25 14:56:10.398378: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
n_epoch = 50
batch_size = 64
taux_validation = 0.1
num_classes = 100
n_images = 50000 # Pour l'entrainement, et 10000 pour le test

In [3]:
validation_size = int(n_images * taux_validation)
train_size = n_images - validation_size

In [4]:
!mc cp s3/afeldmann/projet_cnam/modele_enseignant.keras /home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras
model_enseignant = Sequential([
    Input((224,224,3)),
    ResNet50V2(include_top=False, weights='imagenet', pooling="avg"),
    Dropout(0.25),
    Dense(256, activation="sigmoid", kernel_regularizer = tf.keras.regularizers.L1(0.001)),
    Dropout(0.5),
    Dense(num_classes, activation="softmax", kernel_regularizer = tf.keras.regularizers.L2(0.001))
])
# Keras 3.1.1 est buggé et le chargement direct ne marche pas ici, même si les poids sont bien enregistrés
model_enseignant.load_weights("/home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras")

model_enseignant.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

...nant.keras: 135.73 MiB / 135.73 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 61.11 MiB/s 2s

2024-04-25 14:56:19.143443: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13290 MB memory:  -> device: 0, name: NVIDIA A2, pci bus id: 0000:17:00.0, compute capability: 8.6


In [5]:
def preprocessing(image, label):
    image = tf.image.resize(image, (224, 224))
    label = tf.squeeze(tf.one_hot(label, depth = num_classes), axis = 0)
    return  image, label

augmentation_donnees_keras = Sequential([
    RandomFlip("horizontal"),
    RandomTranslation(0.2,0.2),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomContrast(0.2),
    RandomBrightness(0.2,value_range=(0,1))
])

def augmentation_donnees(image, label):
    return augmentation_donnees_keras(image/255.0, training = True)*255.0, label

def preprocess_resnet(image, label):
    return preprocess_input(image), label

def train_val_split(train_dataset, validation_size):
    X_train, y_train = train_dataset
    return (X_train[:train_size,...], y_train[:train_size,...]), (X_train[train_size:,...], y_train[train_size:,...])


def load_cifar_train_val():
    train_dataset, _ = cifar100.load_data()
    
    train_dataset, validation_dataset = train_val_split(train_dataset, validation_size)
    
    validation_dataset = tf.data.Dataset.from_tensor_slices(validation_dataset).map(preprocessing).batch(batch_size).map(preprocess_resnet).cache().prefetch(tf.data.AUTOTUNE)
    train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset).map(preprocessing).cache().repeat().shuffle(train_size).batch(batch_size).map(augmentation_donnees, num_parallel_calls = tf.data.AUTOTUNE).map(preprocess_resnet, num_parallel_calls = tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
    return train_dataset, validation_dataset

In [6]:
def get_modele_logits(modele):
    config = modele.layers[-1].get_config()
    config['activation'] = linear
    config['name'] = 'logits'
    res = Model(inputs=modele.inputs, outputs=[Dense(**config)(modele.layers[-2].output)])
    res.layers[-1].set_weights([x.numpy() for x in modele.layers[-1].weights])
    res.compile(metrics=['accuracy'])
    return res

@tf.function
def compte_bons(x,y):
    return tf.reduce_sum(tf.cast(tf.equal(tf.argmax(x, axis = 1), tf.argmax(y, axis = 1)), tf.float32))

@tf.function
def softmax(logits, temp):
    expo = tf.exp(logits / temp)
    return expo / tf.reduce_sum(expo, axis = 1, keepdims=True)

@tf.function
def ce(x, y_logits, temp):
    return softmax_cross_entropy_with_logits(x, y_logits / temp) * temp**2

def init_csv_log(fichier):
    with open(fichier,'w') as file:
        file.write("epoch, accuracy_etu, accuracy_dis, val_accuracy_etu, val_accuracy_dis\n")
def append_csv_log(fichier, epoch, accuracy_etu, accuracy_dis, val_accuracy_etu, val_accuracy_dis):
    with open(fichier,'a') as file:
        file.write(f"{epoch:d},{accuracy_etu:.2f},{accuracy_dis:.2f},{val_accuracy_etu:.2f},{val_accuracy_dis:.2f}\n")

def forward_backward_pass_impl(train_dataset_iter, etudiant_logit_model, alpha, temp, optim):
    X_batch, y_batch, enseignant_estim_softmax = next(train_dataset_iter)
    with tf.GradientTape() as tape:
        etudiant_estim_logit = etudiant_logit_model(X_batch, training = True)
        perte = alpha * softmax_cross_entropy_with_logits(y_batch,etudiant_estim_logit) + (1-alpha) * ce(enseignant_estim_softmax,etudiant_estim_logit, temp)
    grads = tape.gradient(perte, etudiant_logit_model.trainable_variables)
    optim.apply_gradients(zip(grads, etudiant_logit_model.trainable_variables))
    return compte_bons(etudiant_estim_logit,y_batch), compte_bons(etudiant_estim_logit, enseignant_estim_softmax)

@tf.function
def val_accuracies(etudiant, enseignant, validation_dataset):
    val_bons_etu, val_bons_dis = 0.0, 0.0
    for X_batch, y_batch in validation_dataset:
        etu_prev = etudiant(X_batch, training = False)
        ens_prev = enseignant(X_batch, training = False)
        val_bons_etu += compte_bons(etu_prev, y_batch)
        val_bons_dis += compte_bons(etu_prev, ens_prev)
    nb_obs_val = tf.cast((validation_size//batch_size) * batch_size, tf.float32)
    val_accuracy_etu = val_bons_etu / nb_obs_val
    val_accuracy_dis = val_bons_dis / nb_obs_val
    return val_accuracy_etu, val_accuracy_dis

def distillateur_kl(etudiant, enseignant, train_dataset, validation_dataset, temp, nom_modele, n_epoch, alpha):
    etudiant_logit_model = get_modele_logits(etudiant)
    enseignant_logit_model = get_modele_logits(enseignant)
    optim = SGD(learning_rate=0.001)
    init_csv_log(f"sauvegardes/{nom_modele}_logs.csv")
    print("C'est parti pour la distillation !\n")
    val_accuracy_etu_max, val_accuracy_dis_max  = 0, 0
    train_dataset_iter = iter(
        train_dataset
        .map(lambda images, label: (images, label, softmax(enseignant_logit_model(images, training = False), temp)), num_parallel_calls = tf.data.AUTOTUNE)
        .prefetch(tf.data.AUTOTUNE)
    )
    forward_backward_pass = tf.function(forward_backward_pass_impl)
    # La tf.function ne peut être que locale car son graphe dépend d'étudiant_logit_model et sinon Tensorflow renvoie une erreur à deux applications successives
    for epoch in range(n_epoch):
        print(f"Époque {epoch + 1} / {n_epoch}")
        n_batch = train_size//batch_size
        barre_progression = Progbar(n_batch, stateful_metrics = ["acc (etu, train)", "acc (dis, train)"])
        bons_epoque_etu, bons_epoque_dis = 0, 0
        for i in range(n_batch):
            bons_etu, bons_dis = forward_backward_pass(train_dataset_iter, etudiant_logit_model, alpha, temp, optim)
            bons_epoque_etu += bons_etu.numpy()
            bons_epoque_dis += bons_dis.numpy()
            n_observ = (i+1) * batch_size
            accuracy_etu, accuracy_dis = bons_epoque_etu / n_observ, bons_epoque_dis / n_observ
            barre_progression.update(i + 1, values = [("acc (etu, train)", accuracy_etu), ("acc (dis, train)", accuracy_dis)])
        val_accuracy_etu, val_accuracy_dis = val_accuracies(etudiant, enseignant, validation_dataset)
        if val_accuracy_etu > val_accuracy_etu_max:
            val_accuracy_etu_max = val_accuracy_etu
            etudiant.save(f"sauvegardes/{nom_modele}_checkpoint.keras")
        if val_accuracy_dis > val_accuracy_dis_max:
            val_accuracy_dis_max = val_accuracy_dis
        if epoch + 1 == 40:
            optim.learning_rate.assign(0.0001)
        if epoch + 1 in [10,20,50]:
            print(f"---> Epoque {epoch + 1:d} - Max val accuracy -> etu : {val_accuracy_etu_max:.4f} | dis : {val_accuracy_dis_max:.4f}")
        append_csv_log(f"sauvegardes/{nom_modele}_logs.csv", epoch, accuracy_etu, accuracy_dis, val_accuracy_etu, val_accuracy_dis)
        print(f"Accuracy (etu, val) : {val_accuracy_etu:.4f} | Accuracy (dis, val) : {val_accuracy_dis:.4f}")

In [7]:
def distillation_resnet18(temp, alpha):
    clear_session()
    train_dataset, validation_dataset = load_cifar_train_val()
    modele = new_modele_resnet()
    nom_modele =  f"model_etudiant_t{temp:d}_a{int(alpha*100):d}"
    distillateur_kl(modele, model_enseignant, train_dataset, validation_dataset, temp, nom_modele, n_epoch, alpha)
    wd = os.getcwd()
    os.system(f"cp {wd}/sauvegardes/{nom_modele}_checkpoint.keras {wd}/sauvegardes/{nom_modele}.keras")
    os.system(f"mc cp {wd}/sauvegardes/{nom_modele}.keras s3/afeldmann/projet_cnam/{nom_modele}.keras")
    os.system(f"mc cp {wd}/sauvegardes/{nom_modele}_logs.csv s3/afeldmann/projet_cnam/{nom_modele}_logs.csv")

In [8]:
def ResNet18():
    resnet18, preprocess_input = Classifiers.get('resnet18')
    resnet = resnet18((224, 224, 3), weights='imagenet', include_top=False)
    resnet_output = GlobalAveragePooling2D()(resnet.output)
    resnet = Model(inputs=resnet.input, outputs=resnet_output)
    return resnet

def new_modele_resnet():
    model = Sequential([
        Input((224,224,3)),
        ResNet18(),
        Dropout(0.25),
        Dense(256, activation="sigmoid", kernel_regularizer = tf.keras.regularizers.L1(0.001)),
        Dropout(0.5),
        Dense(num_classes, activation="softmax", kernel_regularizer = tf.keras.regularizers.L2(0.001))
    ])
    model.compile(metrics=['accuracy'])
    return model

In [9]:
distillation_resnet18(1,1) # témoin

C'est parti pour la distillation !

Époque 1 / 50


2024-04-25 14:57:06.917516: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900


703/703 ━━━━━━━━━━━━━━━━━━━━ 439s 554ms/step - acc (etu, train): 0.0938 - acc (dis, train): 0.0970
Accuracy (etu, val) : 0.1040 | Accuracy (dis, val) : 0.1032
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 334s 475ms/step - acc (etu, train): 0.2666 - acc (dis, train): 0.2716
Accuracy (etu, val) : 0.3159 | Accuracy (dis, val) : 0.3183
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 334s 476ms/step - acc (etu, train): 0.3744 - acc (dis, train): 0.3810
Accuracy (etu, val) : 0.4243 | Accuracy (dis, val) : 0.4259
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 338s 481ms/step - acc (etu, train): 0.4336 - acc (dis, train): 0.4456
Accuracy (etu, val) : 0.4559 | Accuracy (dis, val) : 0.4595
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 339s 483ms/step - acc (etu, train): 0.4806 - acc (dis, train): 0.4927
Accuracy (etu, val) : 0.5000 | Accuracy (dis, val) : 0.5084
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 335s 476ms/step - acc (etu, train): 0.5080 - acc (dis, train): 0.5195
Accuracy (etu, val) : 0.4974 | Accur

In [10]:
distillation_resnet18(1,0)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 414s 560ms/step - acc (etu, train): 0.1035 - acc (dis, train): 0.1062
Accuracy (etu, val) : 0.0917 | Accuracy (dis, val) : 0.0905
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 338s 480ms/step - acc (etu, train): 0.2863 - acc (dis, train): 0.2949
Accuracy (etu, val) : 0.3409 | Accuracy (dis, val) : 0.3444
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 337s 480ms/step - acc (etu, train): 0.3880 - acc (dis, train): 0.3992
Accuracy (etu, val) : 0.3924 | Accuracy (dis, val) : 0.3968
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 334s 475ms/step - acc (etu, train): 0.4477 - acc (dis, train): 0.4607
Accuracy (etu, val) : 0.4469 | Accuracy (dis, val) : 0.4511
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 337s 479ms/step - acc (etu, train): 0.4867 - acc (dis, train): 0.5023
Accuracy (etu, val) : 0.4852 | Accuracy (dis, val) : 0.4932
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 339s 482ms/step - acc (etu, train): 0.5192 - acc (dis, t

In [11]:
distillation_resnet18(3,0)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 427s 581ms/step - acc (etu, train): 0.0687 - acc (dis, train): 0.0701
Accuracy (etu, val) : 0.0873 | Accuracy (dis, val) : 0.0879
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 346s 492ms/step - acc (etu, train): 0.1966 - acc (dis, train): 0.2019
Accuracy (etu, val) : 0.2710 | Accuracy (dis, val) : 0.2744
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 332s 472ms/step - acc (etu, train): 0.3040 - acc (dis, train): 0.3125
Accuracy (etu, val) : 0.3470 | Accuracy (dis, val) : 0.3550
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 331s 471ms/step - acc (etu, train): 0.3725 - acc (dis, train): 0.3820
Accuracy (etu, val) : 0.4101 | Accuracy (dis, val) : 0.4181
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 341s 485ms/step - acc (etu, train): 0.4328 - acc (dis, train): 0.4447
Accuracy (etu, val) : 0.4539 | Accuracy (dis, val) : 0.4633
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 335s 477ms/step - acc (etu, train): 0.4747 - acc (dis, t

In [12]:
distillation_resnet18(8,0)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 420s 570ms/step - acc (etu, train): 0.0380 - acc (dis, train): 0.0396
Accuracy (etu, val) : 0.0551 | Accuracy (dis, val) : 0.0561
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 336s 478ms/step - acc (etu, train): 0.0968 - acc (dis, train): 0.0982
Accuracy (etu, val) : 0.1659 | Accuracy (dis, val) : 0.1655
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 335s 477ms/step - acc (etu, train): 0.1502 - acc (dis, train): 0.1540
Accuracy (etu, val) : 0.2230 | Accuracy (dis, val) : 0.2244
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 338s 481ms/step - acc (etu, train): 0.2015 - acc (dis, train): 0.2066
Accuracy (etu, val) : 0.2364 | Accuracy (dis, val) : 0.2380
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 340s 484ms/step - acc (etu, train): 0.2497 - acc (dis, train): 0.2561
Accuracy (etu, val) : 0.2873 | Accuracy (dis, val) : 0.2909
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 338s 480ms/step - acc (etu, train): 0.2949 - acc (dis, t

In [13]:
distillation_resnet18(1,0.1)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 400s 539ms/step - acc (etu, train): 0.0982 - acc (dis, train): 0.1000
Accuracy (etu, val) : 0.0533 | Accuracy (dis, val) : 0.0529
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 322s 458ms/step - acc (etu, train): 0.2794 - acc (dis, train): 0.2860
Accuracy (etu, val) : 0.3283 | Accuracy (dis, val) : 0.3339
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 322s 458ms/step - acc (etu, train): 0.3789 - acc (dis, train): 0.3900
Accuracy (etu, val) : 0.3898 | Accuracy (dis, val) : 0.3986
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 323s 459ms/step - acc (etu, train): 0.4425 - acc (dis, train): 0.4560
Accuracy (etu, val) : 0.4409 | Accuracy (dis, val) : 0.4475
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 324s 461ms/step - acc (etu, train): 0.4873 - acc (dis, train): 0.5017
Accuracy (etu, val) : 0.4619 | Accuracy (dis, val) : 0.4688
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 329s 468ms/step - acc (etu, train): 0.5152 - acc (dis, t

In [14]:
distillation_resnet18(3,0.1)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 454s 619ms/step - acc (etu, train): 0.0716 - acc (dis, train): 0.0726
Accuracy (etu, val) : 0.0437 | Accuracy (dis, val) : 0.0437
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 347s 493ms/step - acc (etu, train): 0.2044 - acc (dis, train): 0.2100
Accuracy (etu, val) : 0.2410 | Accuracy (dis, val) : 0.2452
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 339s 482ms/step - acc (etu, train): 0.3135 - acc (dis, train): 0.3217
Accuracy (etu, val) : 0.3163 | Accuracy (dis, val) : 0.3211
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 342s 487ms/step - acc (etu, train): 0.3880 - acc (dis, train): 0.3992
Accuracy (etu, val) : 0.3796 | Accuracy (dis, val) : 0.3840
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 341s 485ms/step - acc (etu, train): 0.4449 - acc (dis, train): 0.4585
Accuracy (etu, val) : 0.4339 | Accuracy (dis, val) : 0.4409
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 340s 483ms/step - acc (etu, train): 0.4826 - acc (dis, t

In [15]:
distillation_resnet18(8,0.1)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 427s 579ms/step - acc (etu, train): 0.0416 - acc (dis, train): 0.0425
Accuracy (etu, val) : 0.0527 | Accuracy (dis, val) : 0.0539
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 340s 483ms/step - acc (etu, train): 0.1167 - acc (dis, train): 0.1189
Accuracy (etu, val) : 0.1819 | Accuracy (dis, val) : 0.1881
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 336s 478ms/step - acc (etu, train): 0.1945 - acc (dis, train): 0.1991
Accuracy (etu, val) : 0.2462 | Accuracy (dis, val) : 0.2552
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 343s 488ms/step - acc (etu, train): 0.2688 - acc (dis, train): 0.2760
Accuracy (etu, val) : 0.2957 | Accuracy (dis, val) : 0.3035
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 342s 487ms/step - acc (etu, train): 0.3237 - acc (dis, train): 0.3339
Accuracy (etu, val) : 0.3450 | Accuracy (dis, val) : 0.3522
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 343s 488ms/step - acc (etu, train): 0.3703 - acc (dis, t

In [16]:
distillation_resnet18(1,0.25)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 423s 575ms/step - acc (etu, train): 0.0959 - acc (dis, train): 0.0979
Accuracy (etu, val) : 0.0968 | Accuracy (dis, val) : 0.0958
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 332s 472ms/step - acc (etu, train): 0.2680 - acc (dis, train): 0.2740
Accuracy (etu, val) : 0.3482 | Accuracy (dis, val) : 0.3530
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 324s 461ms/step - acc (etu, train): 0.3783 - acc (dis, train): 0.3894
Accuracy (etu, val) : 0.4393 | Accuracy (dis, val) : 0.4443
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 325s 462ms/step - acc (etu, train): 0.4392 - acc (dis, train): 0.4526
Accuracy (etu, val) : 0.4708 | Accuracy (dis, val) : 0.4764
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 325s 463ms/step - acc (etu, train): 0.4851 - acc (dis, train): 0.4997
Accuracy (etu, val) : 0.4998 | Accuracy (dis, val) : 0.5098
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 324s 461ms/step - acc (etu, train): 0.5131 - acc (dis, t

In [17]:
distillation_resnet18(3,0.25)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 418s 569ms/step - acc (etu, train): 0.0816 - acc (dis, train): 0.0831
Accuracy (etu, val) : 0.0998 | Accuracy (dis, val) : 0.1004
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 329s 468ms/step - acc (etu, train): 0.2304 - acc (dis, train): 0.2360
Accuracy (etu, val) : 0.2959 | Accuracy (dis, val) : 0.2999
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 335s 477ms/step - acc (etu, train): 0.3439 - acc (dis, train): 0.3519
Accuracy (etu, val) : 0.3712 | Accuracy (dis, val) : 0.3772
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 329s 468ms/step - acc (etu, train): 0.4146 - acc (dis, train): 0.4261
Accuracy (etu, val) : 0.4347 | Accuracy (dis, val) : 0.4433
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 329s 468ms/step - acc (etu, train): 0.4628 - acc (dis, train): 0.4745
Accuracy (etu, val) : 0.4894 | Accuracy (dis, val) : 0.4968
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 328s 467ms/step - acc (etu, train): 0.4993 - acc (dis, t

In [18]:
distillation_resnet18(8,0.25)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 412s 554ms/step - acc (etu, train): 0.0541 - acc (dis, train): 0.0551
Accuracy (etu, val) : 0.0619 | Accuracy (dis, val) : 0.0615
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 335s 476ms/step - acc (etu, train): 0.1598 - acc (dis, train): 0.1621
Accuracy (etu, val) : 0.2422 | Accuracy (dis, val) : 0.2452
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 334s 475ms/step - acc (etu, train): 0.2524 - acc (dis, train): 0.2584
Accuracy (etu, val) : 0.3041 | Accuracy (dis, val) : 0.3059
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 333s 474ms/step - acc (etu, train): 0.3309 - acc (dis, train): 0.3387
Accuracy (etu, val) : 0.3694 | Accuracy (dis, val) : 0.3730
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 336s 478ms/step - acc (etu, train): 0.3838 - acc (dis, train): 0.3938
Accuracy (etu, val) : 0.4067 | Accuracy (dis, val) : 0.4079
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 338s 481ms/step - acc (etu, train): 0.4327 - acc (dis, t

In [ ]:
distillation_resnet18(1,0.5)

C'est parti pour la distillation !

Époque 1 / 50
241/703 ━━━━━━━━━━━━━━━━━━━━ 6:02 784ms/step - acc (etu, train): 0.0316 - acc (dis, train): 0.0322

In [ ]:
distillation_resnet18(3,0.5)

In [ ]:
distillation_resnet18(8,0.5)